In [2]:
import pandas as pd
import geopandas as gpd
import datetime
import os

In [3]:
#reading the file

arquivo = "data/us-post-offices-random-coords.csv"

us_post_offices_raw = pd.read_csv(arquivo, delimiter=",")

us_post_offices_raw

C:\Users\AnaLíviadosSantosMaz\AppData\Local\Temp\ipykernel_9228\2520146644.py:5: DtypeWarning: Columns (5,6,11) have mixed types. Specify dtype option on import or set low_memory=False.
  us_post_offices_raw = pd.read_csv(arquivo, delimiter=",")


,Name,AltName,OrigName,State,County1,County2,County3,OrigCounty,Established,Discontinued,...,GNIS.Feature.Class,GNIS.OrigName,GNIS.OrigCounty,GNIS.Latitude,GNIS.Longitude,GNIS.ELEV_IN_M,GNIS.Dist,Latitude,Longitude,RandomCoordsFlag
0,A AND M COLLEGE,NaN,A AND M COLLEGE,MS,OKTIBBEHA,NaN,NaN,Oktibbeha,1922.0,1932.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.508712,-88.703363,True
1,A. B. C.,NaN,A. B. C.,TN,SUMNER,NaN,NaN,Sumner,1880.0,1914.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.419931,-86.563870,True
2,AARON,NaN,AARON,MO,BATES,NaN,NaN,Bates,1895.0,1933.0,...,Post Office,Aaron Post Office (historical),Bates,38.422222,-94.154167,268.0,1.0,38.422222,-94.154167,False
3,AARON,NaN,AARON,TN,BENTON,NaN,NaN,Benton,1889.0,1890.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.205923,-88.094459,True
4,AARON,NaN,AARON,TN,BRADLEY,NaN,NaN,Bradley,1860.0,1860.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.143874,-84.931121,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166135,TRUE,NaN,TRUE,OK,HARMON,NaN,NaN,Harmon,1901.0,1905.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.738756,-99.782970,True
166136,TRUE,NaN,TRUE,TX,YOUNG,NaN,NaN,Young,1894.0,1912.0,...,Populated Place,True,Young,33.279830,-98.728670,359.0,1.0,33.279830,-98.728670,False
166137,SIERRA BLANCA,NaN,SIERRA BLANCA,TX,HUDSPETH,NaN,NaN,Hudspeth,1882.0,NaN,...,Post Office,Sierra Blanca Post Office,Hudspeth,31.175930,-105.360000,1380.0,1.0,31.175930,-105.360000,False
166138,GRAHAM'S MILLS,GRAHAMS MILLS,GRAHAM'S MILLS,TX,SHELBY,NaN,NaN,Shelby,1854.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.962916,-94.393446,True


In [ ]:
# Filtering out the Post Office with ID 127138 (Bexar, Texas), as it appears to be an outlier.
# The recorded establishment year is 1779, which is unusual since Texas was not part of the US at that time.
# This data point is likely incorrect and does not align with the historical population expansion
# and establishment of Post Offices in the US during that period.

us_post_offices_raw = us_post_offices_raw[us_post_offices_raw['ID'] != 127138]

In [4]:
us_post_offices_raw = us_post_offices_raw.dropna(subset=['Established'])

In [5]:
# getting only the columns that are needed

us_post_offices_raw_cleaned = us_post_offices_raw.loc[:,['Name','GNIS.State', 'State', 'County1', 'GNIS.OrigCounty','Established','Discontinued','Continuous','Duration','Latitude','Longitude','RandomCoordsFlag']]

us_post_offices_raw_cleaned

,Name,GNIS.State,State,County1,GNIS.OrigCounty,Established,Discontinued,Continuous,Duration,Latitude,Longitude,RandomCoordsFlag
0,A AND M COLLEGE,NaN,MS,OKTIBBEHA,NaN,1922.0,1932.0,True,10.0,33.508712,-88.703363,True
1,A. B. C.,NaN,TN,SUMNER,NaN,1880.0,1914.0,True,34.0,36.419931,-86.563870,True
2,AARON,MO,MO,BATES,Bates,1895.0,1933.0,True,38.0,38.422222,-94.154167,False
3,AARON,NaN,TN,BENTON,NaN,1889.0,1890.0,True,1.0,36.205923,-88.094459,True
4,AARON,NaN,TN,BRADLEY,NaN,1860.0,1860.0,True,0.0,35.143874,-84.931121,True
...,...,...,...,...,...,...,...,...,...,...,...,...
166135,TRUE,NaN,OK,HARMON,NaN,1901.0,1905.0,True,4.0,34.738756,-99.782970,True
166136,TRUE,TX,TX,YOUNG,Young,1894.0,1912.0,True,18.0,33.279830,-98.728670,False
166137,SIERRA BLANCA,TX,TX,HUDSPETH,Hudspeth,1882.0,NaN,True,NaN,31.175930,-105.360000,False
166138,GRAHAM'S MILLS,NaN,TX,SHELBY,NaN,1854.0,NaN,True,NaN,31.962916,-94.393446,True


In [6]:
# adjusting null values by combining two complementary columns

us_post_offices_raw_cleaned['GNIS.State'] = us_post_offices_raw_cleaned['GNIS.State'].fillna(us_post_offices_raw_cleaned['State'])
us_post_offices_raw_cleaned['GNIS.OrigCounty'] = us_post_offices_raw_cleaned['GNIS.OrigCounty'].fillna(us_post_offices_raw_cleaned['County1'])

us_post_offices_raw_cleaned

,Name,GNIS.State,State,County1,GNIS.OrigCounty,Established,Discontinued,Continuous,Duration,Latitude,Longitude,RandomCoordsFlag
0,A AND M COLLEGE,MS,MS,OKTIBBEHA,OKTIBBEHA,1922.0,1932.0,True,10.0,33.508712,-88.703363,True
1,A. B. C.,TN,TN,SUMNER,SUMNER,1880.0,1914.0,True,34.0,36.419931,-86.563870,True
2,AARON,MO,MO,BATES,Bates,1895.0,1933.0,True,38.0,38.422222,-94.154167,False
3,AARON,TN,TN,BENTON,BENTON,1889.0,1890.0,True,1.0,36.205923,-88.094459,True
4,AARON,TN,TN,BRADLEY,BRADLEY,1860.0,1860.0,True,0.0,35.143874,-84.931121,True
...,...,...,...,...,...,...,...,...,...,...,...,...
166135,TRUE,OK,OK,HARMON,HARMON,1901.0,1905.0,True,4.0,34.738756,-99.782970,True
166136,TRUE,TX,TX,YOUNG,Young,1894.0,1912.0,True,18.0,33.279830,-98.728670,False
166137,SIERRA BLANCA,TX,TX,HUDSPETH,Hudspeth,1882.0,NaN,True,NaN,31.175930,-105.360000,False
166138,GRAHAM'S MILLS,TX,TX,SHELBY,SHELBY,1854.0,NaN,True,NaN,31.962916,-94.393446,True


In [8]:
# filter the DataFrame 'us_post_offices_raw_cleaned' based on two conditions:
us_post_offices_raw_cleaned = us_post_offices_raw_cleaned[
    # condition 1: 'Established' date should be between 1600 and 2000
    ((us_post_offices_raw_cleaned['Established'] >= 1600) & (us_post_offices_raw_cleaned['Established'] <= 2000)) &

    # condition 2: 'Discontinued' date should either be after 1600 or be NaN (not discontinued)
    ((us_post_offices_raw_cleaned['Discontinued'] >= 1600)  | (pd.isnull(us_post_offices_raw_cleaned['Discontinued'])))
]

us_post_offices_raw_cleaned


,Name,GNIS.State,State,County1,GNIS.OrigCounty,Established,Discontinued,Continuous,Duration,Latitude,Longitude,RandomCoordsFlag
0,A AND M COLLEGE,MS,MS,OKTIBBEHA,OKTIBBEHA,1922.0,1932.0,True,10.0,33.508712,-88.703363,True
1,A. B. C.,TN,TN,SUMNER,SUMNER,1880.0,1914.0,True,34.0,36.419931,-86.563870,True
2,AARON,MO,MO,BATES,Bates,1895.0,1933.0,True,38.0,38.422222,-94.154167,False
3,AARON,TN,TN,BENTON,BENTON,1889.0,1890.0,True,1.0,36.205923,-88.094459,True
4,AARON,TN,TN,BRADLEY,BRADLEY,1860.0,1860.0,True,0.0,35.143874,-84.931121,True
...,...,...,...,...,...,...,...,...,...,...,...,...
166135,TRUE,OK,OK,HARMON,HARMON,1901.0,1905.0,True,4.0,34.738756,-99.782970,True
166136,TRUE,TX,TX,YOUNG,Young,1894.0,1912.0,True,18.0,33.279830,-98.728670,False
166137,SIERRA BLANCA,TX,TX,HUDSPETH,Hudspeth,1882.0,NaN,True,NaN,31.175930,-105.360000,False
166138,GRAHAM'S MILLS,TX,TX,SHELBY,SHELBY,1854.0,NaN,True,NaN,31.962916,-94.393446,True


In [9]:
us_post_offices_raw_cleaned = us_post_offices_raw_cleaned.dropna(subset=['Latitude', 'Longitude'])

us_post_offices_raw_cleaned

,Name,GNIS.State,State,County1,GNIS.OrigCounty,Established,Discontinued,Continuous,Duration,Latitude,Longitude,RandomCoordsFlag
0,A AND M COLLEGE,MS,MS,OKTIBBEHA,OKTIBBEHA,1922.0,1932.0,True,10.0,33.508712,-88.703363,True
1,A. B. C.,TN,TN,SUMNER,SUMNER,1880.0,1914.0,True,34.0,36.419931,-86.563870,True
2,AARON,MO,MO,BATES,Bates,1895.0,1933.0,True,38.0,38.422222,-94.154167,False
3,AARON,TN,TN,BENTON,BENTON,1889.0,1890.0,True,1.0,36.205923,-88.094459,True
4,AARON,TN,TN,BRADLEY,BRADLEY,1860.0,1860.0,True,0.0,35.143874,-84.931121,True
...,...,...,...,...,...,...,...,...,...,...,...,...
166135,TRUE,OK,OK,HARMON,HARMON,1901.0,1905.0,True,4.0,34.738756,-99.782970,True
166136,TRUE,TX,TX,YOUNG,Young,1894.0,1912.0,True,18.0,33.279830,-98.728670,False
166137,SIERRA BLANCA,TX,TX,HUDSPETH,Hudspeth,1882.0,NaN,True,NaN,31.175930,-105.360000,False
166138,GRAHAM'S MILLS,TX,TX,SHELBY,SHELBY,1854.0,NaN,True,NaN,31.962916,-94.393446,True


In [10]:
# convert the 'Established' and 'Discontinued' columns to numeric, handling invalid values
us_post_offices_raw_cleaned['Established'] = pd.to_numeric(us_post_offices_raw_cleaned['Established'], errors='coerce')
us_post_offices_raw_cleaned['Discontinued'] = pd.to_numeric(us_post_offices_raw_cleaned['Discontinued'], errors='coerce')

# filter data to include only rows with valid establishment years
data = us_post_offices_raw_cleaned[(us_post_offices_raw_cleaned['Established'] >= 1639) & (us_post_offices_raw_cleaned['Established'] <= 2000)]

# create a list of years for the intervals (1790, 1800, ..., 2000)
years = list(range(1790, 2001, 10))

# initialize a dataframe to store the processed data
processed_data = []

# iterate over each year and filter post offices operational during that year
for year in years:
    subset = data[(data['Established'] <= year) & ((data['Discontinued'] > year) | (data['Discontinued']==0))]
    subset_filtered = subset[['Name', 'Latitude', 'Longitude']].copy()
    subset_filtered['Year'] = year
    subset_filtered['Date'] = pd.to_datetime(subset_filtered['Year'].astype(str) + '-01-01')
    processed_data.append(subset_filtered)

# combine all yearly subsets into one dataframe
processed_data_df = pd.concat(processed_data, ignore_index=True)

processed_data_df

C:\Users\AnaLíviadosSantosMaz\AppData\Local\Temp\ipykernel_9228\807519931.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_post_offices_raw_cleaned['Established'] = pd.to_numeric(us_post_offices_raw_cleaned['Established'], errors='coerce')
C:\Users\AnaLíviadosSantosMaz\AppData\Local\Temp\ipykernel_9228\807519931.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_post_offices_raw_cleaned['Discontinued'] = pd.to_numeric(us_post_offices_raw_cleaned['Discontinued'], errors='coerce')


,Name,Latitude,Longitude,Year,Date
0,BENNINGTON,42.876391,-73.198432,1790,1790-01-01
1,BIDDEFORD,43.491013,-70.456561,1790,1790-01-01
2,BRIDGETOWN EAST,40.685674,-74.384667,1790,1790-01-01
3,ELIZABETHTOWN,40.663992,-74.210701,1790,1790-01-01
4,LITTLE REST,41.480379,-71.522560,1790,1790-01-01
...,...,...,...,...,...
324691,PAGE,38.052890,-81.269830,2000,2000-01-01
324692,SEVEN MILE FORD,36.810390,-81.629560,2000,2000-01-01
324693,SHOWELL,38.399280,-75.210740,2000,2000-01-01
324694,TRAMMEL,37.014550,-82.293750,2000,2000-01-01


In [11]:
# convert the 'Latitude' column to a string type, replace commas with dots, and remove any leading/trailing spaces
processed_data_df['Latitude'] = processed_data_df['Latitude'].astype(str).str.replace(',', '.').str.strip()

# convert the 'Longitude' column to a string type, replace commas with dots, and remove any leading/trailing spaces
processed_data_df['Longitude'] = processed_data_df['Longitude'].astype(str).str.replace(',', '.').str.strip()

# convert the 'Latitude' column to a numeric type, setting invalid parsing to NaN (with errors='coerce')
processed_data_df['Latitude'] = pd.to_numeric(processed_data_df['Latitude'], errors='coerce')

# convert the 'Longitude' column to a numeric type, setting invalid parsing to NaN (with errors='coerce')
processed_data_df['Longitude'] = pd.to_numeric(processed_data_df['Longitude'], errors='coerce') 

# convert the 'Date' column to a datetime format, setting invalid parsing to NaT (with errors='coerce')
processed_data_df['Date'] = pd.to_datetime(processed_data_df['Date'], errors='coerce')

processed_data_df


,Name,Latitude,Longitude,Year,Date
0,BENNINGTON,42.876391,-73.198432,1790,1790-01-01
1,BIDDEFORD,43.491013,-70.456561,1790,1790-01-01
2,BRIDGETOWN EAST,40.685674,-74.384667,1790,1790-01-01
3,ELIZABETHTOWN,40.663992,-74.210701,1790,1790-01-01
4,LITTLE REST,41.480379,-71.522560,1790,1790-01-01
...,...,...,...,...,...
324691,PAGE,38.052890,-81.269830,2000,2000-01-01
324692,SEVEN MILE FORD,36.810390,-81.629560,2000,2000-01-01
324693,SHOWELL,38.399280,-75.210740,2000,2000-01-01
324694,TRAMMEL,37.014550,-82.293750,2000,2000-01-01


In [12]:
# save to CSV
output_path = 'processed_post_offices.csv'
processed_data_df.to_csv(output_path, index=False, encoding='utf-8')

In [14]:
# create a copy of the DataFrame, so the original is not modified
processed_data_df_with_final_discontinued = processed_data_df

# ensure that the 'Date' column is properly converted to datetime format
processed_data_df_with_final_discontinued['Date'] = pd.to_datetime(processed_data_df_with_final_discontinued['Date'])

# create a new column 'year_final' by applying a function that changes the month to December for each date
processed_data_df_with_final_discontinued['year_final'] = processed_data_df_with_final_discontinued['Date'].apply(
    lambda x: x.replace(month=12)
)

processed_data_df_with_final_discontinued


,Name,Latitude,Longitude,Year,Date,year_final
0,BENNINGTON,42.876391,-73.198432,1790,1790-01-01,1790-12-01
1,BIDDEFORD,43.491013,-70.456561,1790,1790-01-01,1790-12-01
2,BRIDGETOWN EAST,40.685674,-74.384667,1790,1790-01-01,1790-12-01
3,ELIZABETHTOWN,40.663992,-74.210701,1790,1790-01-01,1790-12-01
4,LITTLE REST,41.480379,-71.522560,1790,1790-01-01,1790-12-01
...,...,...,...,...,...,...
324691,PAGE,38.052890,-81.269830,2000,2000-01-01,2000-12-01
324692,SEVEN MILE FORD,36.810390,-81.629560,2000,2000-01-01,2000-12-01
324693,SHOWELL,38.399280,-75.210740,2000,2000-01-01,2000-12-01
324694,TRAMMEL,37.014550,-82.293750,2000,2000-01-01,2000-12-01


In [15]:
# update the 'year_final' column to replace each date with a new datetime object
# where the year remains the same, but the date is set to January 1st at noon (12:00 PM)
processed_data_df_with_final_discontinued['year_final'] = processed_data_df_with_final_discontinued['year_final'].apply(
    lambda x: datetime.datetime(x.year, 1, 1, 12, 0, 0)  # create a datetime object with the same year, January 1st, and set time to 12:00 PM
)


In [16]:
processed_data_df_with_final_discontinued

,Name,Latitude,Longitude,Year,Date,year_final
0,BENNINGTON,42.876391,-73.198432,1790,1790-01-01,1790-01-01 12:00:00
1,BIDDEFORD,43.491013,-70.456561,1790,1790-01-01,1790-01-01 12:00:00
2,BRIDGETOWN EAST,40.685674,-74.384667,1790,1790-01-01,1790-01-01 12:00:00
3,ELIZABETHTOWN,40.663992,-74.210701,1790,1790-01-01,1790-01-01 12:00:00
4,LITTLE REST,41.480379,-71.522560,1790,1790-01-01,1790-01-01 12:00:00
...,...,...,...,...,...,...
324691,PAGE,38.052890,-81.269830,2000,2000-01-01,2000-01-01 12:00:00
324692,SEVEN MILE FORD,36.810390,-81.629560,2000,2000-01-01,2000-01-01 12:00:00
324693,SHOWELL,38.399280,-75.210740,2000,2000-01-01,2000-01-01 12:00:00
324694,TRAMMEL,37.014550,-82.293750,2000,2000-01-01,2000-01-01 12:00:00


In [17]:
processed_data_df_with_final_discontinued.to_csv('point_map.csv',index=False, encoding='utf-8')